
**PROJETO IA - ROTA DE TRÂNSITO IDEAL**





Comandos para instalar as bibliotecas:

-conda install pandas

-conda install haversine

-conda install -c conda-forge folium

**Ou**

-pip install pandas

-pip install folium

-pip install haversine



In [ ]:
!pip install pandas

!pip install folium

!pip install haversine

Fazemos as importações necessárias

In [ ]:
import requests
import json
import pandas as pd
import folium
import time
import random
from haversine import haversine
from folium.features import DivIcon

Criamos strings contendo os endereços de partida e destino

In [ ]:
Pontos = []

Pontos.append({'Endereço':'Rua Minas Gerais, 440-554 - Guaíra', 'NomeLugar':'Casa do inicio'})
Pontos.append({'Endereço':'Rua Jaime Rodrigues da Rocha, 2-218 - Capão Raso', 'NomeLugar':'Casa da namorada'})
Pontos.append({'Endereço':'Rua Bernardo Jacintho da Veiga, 656-824 - Novo Mundo, Curitiba - PR, 81050-100', 'NomeLugar':'Casa do amigo'})
Pontos.append({'Endereço':'R. Curupis, 628-742 - Santa Quiteria', 'NomeLugar':'Casa da amiga'})
Pontos.append({'Endereço':'Av. Pres. Kennedy, 4121 Portão Curitiba PR', 'NomeLugar':'Shooping'})

Realizamos requisições na API MapBox, para obter as coordenadas dos endereços

In [ ]:
req = []
for ponto in Pontos:
  req.append(requests.get('https://api.mapbox.com/geocoding/v5/mapbox.places/' + ponto['Endereço'] + '.json?access_token=pk.eyJ1IjoiamVhbmN3YiIsImEiOiJja2VnZmI2czYwa3loMnFyejdhaDVzZzhjIn0.5YA3in_HTwmOM1WOa7CTIg&bbox=-73.9872354804, -33.7683777809, -34.7299934555, 5.24448639569'))
  time.sleep(0.5)

Exibimos o retorno da API, para nos certificar que o endereço retornou corretamente

In [ ]:
for r in req:
  print(r.text)

Transformamos o retorno em um dicionário Python

In [ ]:
jsonEnds = []
for r in req:
  jsonEnds.append(json.loads(r.text))

Convertemos as coordenadas presentes no retorno em uma lista de coordenadas

In [ ]:
coordsEnde = []
for jsonEnd in jsonEnds:
  coordsEnde.append(jsonEnd['features'][0]['geometry']['coordinates'])

Criamos strings com as coordenadas dos endereços, conversão necessária para fazer requisições na API do TomTom (que retornará as rotas)

In [ ]:
coord_str = []
for coord in coordsEnde:
  coord_str.append(str(coord[1]) + '%2C' + str(coord[0]))

GET na API do tomtom e tratamento do retorno (gerando uma lista com as rotas possíveis entre os pontos)

In [ ]:
request_routes = []
for index1, coord1 in enumerate(coord_str):
  if index1 != (len(coord_str) - 1):
    for index2, coord2 in enumerate(coord_str):
      if index2 != 0:
        routes_list = []
        if coord1 != coord2:
          r = requests.get('https://api.tomtom.com/routing/1/calculateRoute/' + coord1 + '%3A' + coord2 + '/json?maxAlternatives=3&avoid=unpavedRoads&key=fZJ66W6Qzj9AMLpz1YItfHRs8IBzRi1z')
          time.sleep(0.5)
          jsonDict = json.loads(r.text)
          for route in jsonDict['routes']:
            routes_dict = {}
            routes_dict['travelTime'] = route['summary']['travelTimeInSeconds']
            routes_dict['points'] = pd.DataFrame(route['legs'][0]['points'])
            routes_list.append(routes_dict)
          random.shuffle(routes_list) #Misturando a lista para dificultar um pouco...
          path_orig = index1
          path_dest = index2
          path_dict = {'path_orig':index1, 'path_dest':index2, 'routes_list':routes_list}
          request_routes.append(path_dict)

Printamos para validar

In [ ]:
print(request_routes)

Retornamos 4 rotas possíveis entre cada ponto, agora iremos filtrar entre essas rotas apena as melhores (menos gasto e tempo). Feito isso criamos uma lista com as rotas ideais a serem usadas pela I.A

In [ ]:
kml = 6 #COLOCAR GASTO KM/L DO CARRO
routesI = []
for path in request_routes: #Passando por todas as rotas, de todos os pontos
  current_node = path['path_orig']
  decI = 0
  indexRouteI = 0
  for indexRoute, route in enumerate(path['routes_list']):
    distFinal = 0
    routePoints = []
    for index, row in route['points'].iterrows():
        if index < (len(route['points']) - 1):
          pA = (row['latitude'], row['longitude'])
          nextIndex = index + 1
          pB = (route['points']['latitude'][nextIndex], route['points']['longitude'][nextIndex])
          distFinal += haversine(pA, pB) #Chamando equação de haversine, calcula distância entre duas coordenadas
        routePoints.append([row['latitude'], row['longitude']])
    gasto = round(distFinal * kml, 0)
    pesoDec = (gasto * route['travelTime']) #Multiplicando os dois fatores para ter um peso de decisão
    if decI == 0 or decI > pesoDec:
      decI = pesoDec #Se estiver entre as rotas ideais, salva
      pointsI = routePoints 
  edge = {'dest_node':path['path_dest'], 'edge_cost': decI, 'route_points':pointsI}
  found_node = False
  for routeIndex, routeI in enumerate(routesI): #Passando pela lista de rotas ideais
    if routeI['orig_node'] == current_node: #Se achar um item na lista cujo nó de origem é igual a origem atual, usa ele
      routesI[routeIndex]['dest'].append(edge) #Coloca o destino
      found_node = True
  if not found_node: #Senão encontrar o nó origem atual (ponto) na lista de rotas ideias, cria ele e coloca um ponto de destino
    new_routeI = {'orig_node':current_node, 'dest':[]}
    new_routeI['dest'].append(edge)
    routesI.append(new_routeI)

print(routesI)

In [ ]:
len(routesI)

Busca pelo melhor caminho
---



In [ ]:
idealRoutes = routesI
currentPoint = 0
arrivalPoint = 4
lowerValue = 0
iterations = 0
lastPoints = [currentPoint]
path = ''
isPassed = False
optRoute = []

def bestPath(currentPoint2):
  global idealRoutes
  global currentPoint
  global arrivalPoint
  global lowerValue
  global iterations
  global lastPoints
  global path
  global isPassed

  vetor = idealRoutes[currentPoint2]['dest']

  for value in vetor:
      if lowerValue == 0 or lowerValue > value['edge_cost']:
        
        for item in lastPoints:
            if item == value['dest_node']:
              isPassed = True

        if value['dest_node'] == arrivalPoint and len(lastPoints) != len(idealRoutes):
            continue
        
        if len(lastPoints) == len(idealRoutes) - 1:
            currentPoint = arrivalPoint
        
        if isPassed == False:
            lowerValue = value['edge_cost'];
            currentPoint = value['dest_node'];

      isPassed = False

  optRoute.append(currentPoint)
  path = path + str(currentPoint) + '->'
  
  lastPoints.append(currentPoint)
  print(path)

  if iterations >= len(idealRoutes) or currentPoint == arrivalPoint: # Segunda condição apenas para não ocorrer erros.
      return
  else:
      iterations = iterations + 1
      lowerValue = 0
      bestPath(currentPoint)

bestPath(currentPoint)



In [ ]:
coordsRoute = []
currentNode = 0
for idx, route in enumerate(optRoute):
  for edge in routesI:
    if edge['orig_node'] == currentNode:
        nextNode = optRoute[idx]
        print(currentNode)
        for destNode in edge['dest']:
          if destNode['dest_node'] == nextNode:
            print(destNode)
            for coords in destNode['route_points']:
              coordsRoute.append(coords)
              currentNode = nextNode
        break
print(coordsRoute)

Criamos um objeto mapa da biblioteca Folium, será usado para exibição

In [ ]:
m = folium.Map(location=[float(coordsRoute[0][0]), float(coordsRoute[0][1])],
               zoom_start=13)

Criamos um marcador no mapa no ponto de partida e chegada, para facilitar a visualização

In [ ]:
for index, coord in enumerate(coordsEnde):
  print(coord)
  folium.Marker([coord[1], coord[0]], 
                popup='<b>' + Pontos[index]['NomeLugar'] + '</b>', 
                icon=DivIcon(icon_size=(200,36),
                  icon_anchor=(0,0),
                  html='<div style="font-size: 12pt; color:blue; font-weight: bold">%s</div>' % Pontos[index]['NomeLugar'],
                )).add_to(m)
  folium.Marker([coord[1], coord[0]], 
                popup='<b>' + Pontos[index]['NomeLugar'] + '</b>').add_to(m)

Percorremos as coordenadas da rota para criar uma lista de tuplas, então utilizamos ela para criar o percurso no mapa

In [ ]:
tuples = [tuple(x) for x in coordsRoute]
folium.PolyLine(tuples, color="red", weight=2.5, opacity=1).add_to(m)

Exibimos o mapa

In [ ]:
m